# Different Optimization approaches to solve the Unit Commitment Problem (UC)

### Table of Contents
- [Introduction to UC](#intro)
- [Three Different Optimization approaches](#opt_appr)
- [Implementation Design](#impl)
- [Abstract base class `UnitCommitment`](#abs)
    - [Decision variables](#abs-vars)
    - [Constraints](#abs-cons)
    - [Objective function](#abs-obj)
- [A Deterministic Optimization approach to UC](#det)
- [A Robust Optimization approach to UC](#rob)
    - [Decision variables modeling the uncertainty of renewable energy production](#rob-vars)
    - [Constraints modeling renewable energy variability in demand coverage and system reserve dimensioning](#rob-cons)
    - [Business interpretation](#rob-bus)
- [A Stochastic Optimization Approach to UC](#sto)
    - [Decision variables modeling the uncertainty of renewable energy production](#sto-vars)
    - [Constraints modeling renewable energy variability in demand coverage and system reserve dimensioning](#sto-cons)
    - [Penalizing projected cost of reserve in the objective function](#sto-obj)
    - [Business interpretation](#sto-bus)

## Introduction to UC

<a id=intro></a>

UC refers to a family of optimization problems, which are usually modeled and solved with Mixed-Integer Programming techniques. In real-world applications, UC provides optimal plans to the electricity production of a given set of (connected) power generators, called *units*, by ensuring that the total electrical energy demand is matched at the minimum operational cost over a targeted time horizon. In this particular use-case, UC is characterized by the following elements:
- __two kinds of units__: *thermal* units (burning coal/fuel) and *renewable* energy units (solar and wind farms)
- __hourly energy load__: hourly *electricity demand* generated by consumers
- __time horizon__: the planning horizon covers one day of operations
- __units' operational requirements__: *operational restrictions* characterizing the units, like physical limitations on the energy production, ramp-up/down variations in consecutive hours, etc.
- __uncertainty__: the renewable energy generation is subject to uncertainty, then it is modeled as a *stochastic unknown* (expected value and standard deviation are given for each hour)
- __energy reserve__: the *energy variation* between the minimum and the maximum total power that the active units can induce in each hour (the energy reserve is crucial as it makes the planning to be conservative respect to the actual deviations occuring during the real-time operations)  


## Three Different Optimization approaches

<a id=opt_appr></a>

This notebook implements three different optimization approaches for UC. Each approach leads to a mixed-integer linear program that is modeled and solved with [IBM$\circledR$ Decision Optimization CPLEX$\circledR$ Modeling for Python](https://developer.ibm.com/docloud/documentation/optimization-modeling/modeling-for-python/): [`docplex`](https://pypi.org/project/docplex/). The three approaches differ in how the renewable energy generation is modeled. In __Deterministic UC formulation__, the renewable energy generation is assumed to be deterministic, hence energy produced by renewable farms (units) is given by constant values. In the following two approaches, the renewable energy generation is affected by uncertainty, then it is modeled as a stochastic unknown. In particular, the __Stochastic UC formulation__ is based on a common stochastic programming technique: given hourly expected values and standard deviations for the renewable generation, a discrete set of (100) simulated scenario is pre-computed under the assumption that their realization is normally distributed. The __Robust UC formulation__ handles uncertainty in a different way: hourly expected values and standard deviations of renewable generation give rise to deterministic uncerntainity intervals, which are used to build a mathematical program according to the Soyster's robust modeling technique (no assumption is required on the probability distribution expressing the realization of the stochastic unknown).  


## Implementation Design

<a id=impl></a>

The three addressed formulations have some common decision variables and constraints, while they mostly differ in how the demand coverage constraints are handled in each optimization approach. For this reason, main idea is adopting an Object-Oriented Programming (OOP) technique that keeps the common part of the three formulations in a common base class, whereas a specialized derived class is defined for each formulation. More in detail, the abstract factory design pattern is applied by exploiting [`abc` (Abstract Base Classes)](https://docs.python.org/2.7/library/abc.html#module-abc) module of Python Standard Library. Python module `abc` allows to define a contract between the abstract base class and its derived classes as doable in other OOP languages, but, additionally, it allows the abstract methods to have a body that can be either overridden or decorated in derived classes.<br>

<span style="color:red; font-family:Courier;">UnitCommitment</span> (abstract base class)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\uparrow$<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\arrowvert\leftarrow$ <span style="color:blue; font-family:Courier">DeterministicUnitCommitment</span> (derived class)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\bracevert$<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\arrowvert\leftarrow$ <span style="color:blue; font-family:Courier">RobustUnitCommitment</span> (derived class)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\bracevert$<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\arrowvert\leftarrow$ <span style="color:blue; font-family:Courier">StochasticUnitCommitment</span> (derived class)<br>

The addressed formulations are discussed below by following the adopted implementation pattern.

## Abstract base class `UnitCommitment`: common decision variables, constraints and objective function

<a id=abs></a>

Let $U := U_H \cup U_W$ be the set of units, where $U_H$ and $U_W$ are respectively the (disjoint) subsets of thermal and renewable energy units. Let $T$ be the set of periods, i.e. the time intervals in which the planning time horizon is discretized. 


### Decision variables

<a id=abs-vars></a>

The following decision variables are introduced.
- energy produced by unit $u$ in period $t$:<br><br> 
$$p_{ut} \in \mathbb{R}_+\qquad \forall u\in U,\, t\in T$$ 
<br>
- status of unit $u$ in period $t$, which can be *on*, *off* or *active* if $u$ is either switched on or switched off in $t$, or it is active since (at least) $t-1$:<br><br>
$$x^{\text{on}}_{ut},\,x^{\text{off}}_{ut},\,x^{\text{active}}_{ut} \in \{0,1\}\qquad \forall u\in U,\, t\in T$$ 
<br>


### Constraints

<a id=abs-cons></a>

The following constraints hold to model operational restrictions of the units or the entire electrical system:
- upper and lower bounds on the energy produced:<br><br>
\begin{align}
p_{ut} &\leq \lambda^{\text{max}}_{ut} \cdot x^{\text{active}}_{ut} &\qquad\forall u\in U,\, t\in T \\
p_{ut} &\geq \lambda^{\text{min}}_{ut} \cdot x^{\text{active}}_{ut} &\qquad\forall u\in U,\, t\in T \\
\sum_{t\in T}p_{ut} &\geq \varepsilon^{\text{min}}_{u} &\qquad\forall u\in U
\end{align}
where $\lambda^{\text{max}}_{ut},\, \lambda^{\text{min}}_{ut},\, \varepsilon^{\text{min}}_{u} \in \mathbb{R}_+$ are system configuration constants. 
- electrical system inertia:<br><br>
\begin{align}
\sum_{u\in U_H}p_{ut} &\leq \pi^{\text{max}}_{t} &\qquad\forall t\in T\\
\sum_{u\in U}x^{\text{active}}_{ut} &\geq \nu^{\text{min}}_{t} &\qquad\forall t\in T
\end{align}
where $\pi^{\text{max}}_t \in \mathbb{R}_+ $ and $\nu^{\text{min}}_{t} \in \mathbb{Z}_+$ are other system configuration constants. 
- scheduled interventions (like maintenance) on units:<br><br>
$$\upsilon^{-}_{ut} \leq x^{\text{active}}_{ut} \leq \upsilon^{+}_{ut} \qquad\forall u\in U,\, t\in T$$
where $\upsilon^{-}_{ut},\, \upsilon^{+}_{ut} \in \{0,1\}$ are configuration constants such that $\upsilon^{-}_{ut} \leq \upsilon^{+}_{ut}$ (in case of scheduled maintenance for $u$ during $t$, $u$ is either excluded or activated respectively if either $\upsilon^{+}_{ut} = 0$ or $\upsilon^{-}_{ut} = 1$).

The following constraints model feasible transitions of unit status over consecutive periods:<br><br>
\begin{align}
x^{\text{on}}_{u0} &= 0 &\qquad\forall u\in U_0^\text{active}\\
x^{\text{off}}_{u0} + x^{\text{active}}_{u0} &= 1 &\qquad\forall u\in U_0^\text{active}\\
x^{\text{off}}_{u0} &= 0 &\qquad\forall u\in U_0^\text{off}\\
x^{\text{on}}_{u0} - x^{\text{active}}_{u0} &= 0 &\qquad\forall u\in U_0^\text{off}\\
x^{\text{active}}_{ut} - x^{\text{active}}_{u,t-1} - x^{\text{on}}_{ut} &\leq 0 &\qquad\forall u\in U,\, t\in T\setminus\{0\}\\
x^{\text{active}}_{u,t-1} - x^{\text{active}}_{ut} + x^{\text{on}}_{ut} - x^{\text{off}}_{ut} &= 0 &\qquad\forall u\in U,\, t\in T\setminus\{0\}
\end{align}
where $U_0^\text{active},\, U_0^\text{off} \subseteq U$ are subsets of units whose status is respectively *active* or *off* at the beginning of time horizon ($t = 0$).

Finally, the following constraints model operational restrictions of thermal units:
- ramp-up and ramp-down restrictions:<br><br>
\begin{align}
-\varrho^-_u &\leq p_{u0} - \pi_{u} &\leq \varrho^+_u &\qquad\forall u\in U_H\\
-\varrho^-_u &\leq p_{ut} - p_{u,t-1} &\leq \varrho^+_u &\qquad\forall u\in U_H,\, t\in T\setminus\{0\}
\end{align}
where $\pi_{u},\, \varrho^-_u,\, \varrho^+_u \in \mathbb{R}_+$ are constants, which represent respectively initial power, ramp-up and ramp-down power variation of each thermal unit.

- up-time and down-time restrictions:<br><br>
\begin{align}
\sum_{\tau = 0}^{\tau^\text{up}} x^{\text{on}}_{u, t - \tau} &\leq x^{\text{active}}_{ut} &\qquad\forall u\in U_H,\, t\in T: t\geq \tau^\text{up}\\
\sum_{\tau = 0}^{\tau^\text{down}} x^{\text{off}}_{u,t - \tau} &\leq 1 - x^{\text{active}}_{ut} &\qquad\forall u\in U_H,\, t\in T: t\geq \tau^\text{down}
\end{align}
where $\tau^\text{up},\, \tau^\text{down} \in \mathbb{N}$ are constants, which express up-time and down-time in number of periods for any thermal unit.

### Objective function

<a id=abs-obj></a>

The objective function aims to minimize the overall production cost:<br><br>
$$\min \, \sum_{u\in U}\sum_{t\in T}\left( c^{v}_{u} \cdot p_{ut} + c^{f}_{u} \cdot x^{\text{active}}_{ut} + c^{s}_u \cdot x^{\text{on}}_{ut} \right)$$

where $c^{p}_{v},\, c^{f}_{u},\, c^{s}_{u}\in \mathbb{R}_+$ are respectively variable, fixed and start-up costs associated to units production.

In [1]:
from abc import abstractmethod, ABCMeta
import pandas as pd
from docplex.mp.environment import Environment
from docplex.mp.model import Model


class UnitCommitment(object):
    __metaclass__ = ABCMeta

    @abstractmethod
    def import_data(self, sc_inputs):
        ### Thermal units
        self.df_thermal_units = sc_inputs['df_thermal_units']
        self.df_thermal_units.set_index('unit', inplace=True)
        self.df_thermal_units_conf = sc_inputs['df_thermal_units_conf']
        self.df_thermal_units_conf.set_index(['unit', 'period'], inplace=True)

        ### Renewable units
        self.df_renewable_units = sc_inputs['df_renewable_units']
        self.df_renewable_units.set_index('unit', inplace=True)
        self.df_renewable_units_conf = sc_inputs['df_renewable_units_conf']
        self.df_renewable_units_conf.set_index(['unit', 'period'], inplace=True)

        ### System properties
        self.df_system_config = sc_inputs['df_system_config']

        #### Demand
        self.df_load = sc_inputs['df_load']
        self.df_load.set_index('period', inplace=True)

    def __build_decision_vars_subset(self, units, periods, name=""):
        if name:
            name = "_{}".format(name)

        # true iff unit u is active at period t
        in_use = self.uc.binary_var_matrix(keys1=units, keys2=periods, name="in_use{}".format(name))
        # true if unit in_use is turned on at period t
        on = self.uc.binary_var_matrix(keys1=units, keys2=periods, name="on{}".format(name))
        # true if unit in_use is switched off at period t
        off = self.uc.continuous_var_matrix(keys1=units, keys2=periods, lb=0, ub=1, name="off{}".format(name))
        # energy production of unit u at period t
        p = self.uc.continuous_var_matrix(keys1=units, keys2=periods, name="p{}".format(name))
        # Organize all decision variables in a DataFrame indexed by 'units' and 'periods'
        df_decision_vars = pd.DataFrame(
            {"in_use{}".format(name): in_use, "on{}".format(name): on, "off{}".format(name): off, \
             "p{}".format(name): p})

        df_decision_vars.index.names = ['unit', 'period']
        return df_decision_vars

    @abstractmethod
    def build_decision_vars(self):
        thrm_units = self.df_thermal_units.index.values.tolist()
        periods = self.df_load.index.values.tolist()
        self.df_thrm_vars = self.__build_decision_vars_subset(thrm_units, periods)

        rnwb_units = self.df_renewable_units.index.values.tolist()
        self.df_rnwb_vars = self.__build_decision_vars_subset(rnwb_units, periods, "rnwb")

    @abstractmethod
    def build_var_expressions(self):
        df_join_thrm_vars_up = self.df_thrm_vars.join(self.df_thermal_units_conf, how='inner')
        thermal_groupby = df_join_thrm_vars_up.groupby(level='period')

        df_join_rnwb_vars_up = self.df_rnwb_vars.join(self.df_renewable_units_conf, how='inner')
        renewable_groupby = df_join_rnwb_vars_up.groupby(level='period')

        periods = self.df_load.index.values.tolist()

        self.p_tot = {}
        self.r_up_tot = {}
        self.r_down_tot = {}

        for period in periods:
            thrm = thermal_groupby.get_group(period)
            rnwb = renewable_groupby.get_group(period)
            self.p_tot[period] = self.uc.sum(thrm.p) + self.uc.sum(rnwb.p_rnwb)
            self.r_up_tot[period] = self.uc.sum(thrm.LmaxR * thrm.in_use - thrm.p) \
                                    + self.uc.sum(rnwb.LmaxR * rnwb.in_use_rnwb - rnwb.p_rnwb)
            self.r_down_tot[period] = self.uc.sum(thrm.p - thrm.LminR * thrm.in_use) \
                                      + self.uc.sum(rnwb.p_rnwb - rnwb.LminR * rnwb.in_use_rnwb)

    def __add_var_limit_cons(self):
        df_join_thrm_vars_up = self.df_thrm_vars.join(self.df_thermal_units_conf, how='inner')
        for item in df_join_thrm_vars_up.itertuples(index=False):
            self.uc.add_constraint(item.p <= item.Lmax * item.in_use)
            self.uc += (item.p >= item.Lmin * item.in_use)
            item.in_use.ub = int(item.Umax)
            item.in_use.lb = int(item.Umin)

        df_join_rnwb_vars_up = self.df_rnwb_vars.join(self.df_renewable_units_conf, how='inner')
        for item in df_join_rnwb_vars_up.itertuples(index=False):
            self.uc.add_constraint(item.p_rnwb <= item.Lmax * item.in_use_rnwb)
            self.uc += (item.p_rnwb >= item.Lmin * item.in_use_rnwb)
            item.in_use_rnwb.ub = int(item.Umax)
            item.in_use_rnwb.lb = int(item.Umin)

    def __add_therm_units_cons(self, init_status=None, active_at_beginning=False):
        periods = self.df_load.index.values.tolist()
        for unit, r in self.df_thrm_vars.groupby(level='unit'):
            # initial status
            if init_status and len(init_status) > 0:
                raise RunTimeError("Fuctionality is not available - To be impemented")
            elif active_at_beginning or (self.df_thermal_units.Init_Power[unit] > 0):
                r.on[0].ub = 0
                self.uc += (r.off[0] + r.in_use[0] == 1)
            else:
                self.uc += (r.on[0] == r.in_use[0])
                r.off[0].ub = 0

            # ramp-up / ramp-down
            u_RampUp = self.df_thermal_units.RampUp[unit]
            u_RampDown = self.df_thermal_units.RampDown[unit]
            u_initial = self.df_thermal_units.Init_Power[unit]
            r.p[0].ub = (u_initial + u_RampUp)
            r.p[0].lb = max(0, u_initial - u_RampDown)
            for (p_curr, p_next) in zip(r.p, r.p[1:]):
                self.uc.add_constraint(p_next - p_curr <= u_RampUp)
                self.uc.add_constraint(p_curr - p_next <= u_RampDown)

            # min uptime / downtime
            minONtime = self.df_thermal_units.minON[unit]
            minOFFtime = self.df_thermal_units.minOFF[unit]
            for t in range(minONtime, len(periods)):
                ctname = "minON_{0}_{1}".format(*r.index[t])
                self.uc.add_constraint(self.uc.sum(r.on[(t - minONtime + 1):(t + 1)]) <= r.in_use[t], ctname)
            for t in range(minOFFtime, len(periods)):
                ctname = "minOFF_{0}_{1}".format(*r.index[t])
                self.uc.add_constraint(self.uc.sum(r.off[(t - minOFFtime + 1):(t + 1)]) <= 1 - r.in_use[t], ctname)

            # on / off / in-use propagation
            for (u_curr, u_next, v_next, w_next) in zip(r.in_use, r.in_use[1:], r.on[1:], r.off[1:]):
                self.uc.add_constraint(u_next - u_curr <= v_next)
                self.uc.add_constraint(u_curr - u_next + v_next == w_next)

    @abstractmethod
    def add_demand_cons(self):
        inde_groupby = self.df_rnwb_vars.groupby(level='period')
        for period, r in self.df_thrm_vars.groupby(level='period'):
            iv = inde_groupby.get_group(period)
            # inertia
            self.uc.add_constraint(self.uc.sum(r.in_use) + self.uc.sum(iv.in_use_rnwb) >= self.df_system_config.minUON[0])
            # demand percentage
            total_demand = self.df_load.D[period]
            for unit in r.index.values.tolist():
                self.uc.add_constraint(r.p[unit] <= self.df_system_config.PmaxD[0] * total_demand)

    def __build_objective_and_kpi_expr(self):
        df_join_obj = self.df_thrm_vars.join(self.df_thermal_units[['c_fixed', 'c_variable', 'c_start']], how='inner')
        df_join_obj_rnwb = self.df_rnwb_vars.join(self.df_renewable_units[['c_fixed', 'c_variable']], how='inner')

        total_a = self.uc.sum(df_join_obj.in_use * df_join_obj.c_fixed)
        total_b = self.uc.sum(df_join_obj.p * df_join_obj.c_variable)
        total_c = self.uc.sum(df_join_obj.on * df_join_obj.c_start)
        total_a_rnwb = self.uc.sum(df_join_obj_rnwb.in_use_rnwb * df_join_obj_rnwb.c_fixed)
        total_b_rnwb = self.uc.sum(df_join_obj_rnwb.p_rnwb * df_join_obj_rnwb.c_variable)

        total_economic_cost = total_a + total_b + total_c
        total_economic_cost_rnwb = total_a_rnwb + total_b_rnwb

        self.obj = total_economic_cost + total_economic_cost_rnwb

        total_nb_used = self.uc.sum(self.df_thrm_vars.in_use)
        total_nb_starts = self.uc.sum(self.df_thrm_vars.on)
        # add KPIs
        self.uc.add_kpi(total_a, "Total Fixed Cost")
        self.uc.add_kpi(total_b, "Total Variable Cost")
        self.uc.add_kpi(total_c, "Total Startup Cost")
        self.uc.add_kpi(total_economic_cost, "Total Economic Cost")
        self.uc.add_kpi(total_nb_used, "Total #used")
        self.uc.add_kpi(total_nb_starts, "Total #starts")
        periods = self.df_load.index.values.tolist()
        for period in periods:
            self.uc.add_kpi(self.p_tot[period], "PROD_TOT_{}".format(period))
            self.uc.add_kpi(self.r_up_tot[period], "RES_UP_TOT_{}".format(period))
            self.uc.add_kpi(self.r_down_tot[period], "RES_DOWN_TOT_{}".format(period))

    @abstractmethod
    def add_objective_and_kpis(self):
        self.__build_objective_and_kpi_expr()
        # minimize total cost
        self.uc.minimize(self.obj)

    @abstractmethod
    def __init__(self, sc_inputs):
        self._solved = False

        # Import data
        self.import_data(sc_inputs)

        # Initialize cplex
        self._env = Environment()
        self.uc = Model("UnitCommitment", log_output=True)

        # Create variables
        self.build_decision_vars()

        # Create expressions
        self.build_var_expressions()

        # Add constraints
        self.__add_var_limit_cons()
        self.__add_therm_units_cons()
        self.add_demand_cons()

        # Add objective and KPIs
        self.add_objective_and_kpis()
        self._sc_outputs = {}

    @staticmethod
    def __prepare_plan_df(df_used, df_started, df_stopped):
        df_plan = df_used.rename(columns={'value': 'active'}).merge(df_started.rename(columns={'value': 'on'}), on=['period', 'unit'])
        df_plan = df_plan.merge(df_stopped.rename(columns={'value': 'off'}), on=['period', 'unit'])
        df_plan['plan'] = df_plan.apply(lambda x: "start" if abs(x.on - 1) < 1.0e-6 else \
            ("stop" if abs(x.off - 1) < 1.0e-6 else \
                 ("active" if abs(x.active - 1) < 1.0e-6 else "idle")), axis=1)
        df_plan.drop(['on', 'off', 'active'], axis=1, inplace=True)
        return df_plan

    @staticmethod
    def __prepare_reserve_kpi_df(kpis_bd):
        def extract_period_reserve_kpi(kpi_name):
            kpi_df = kpis_bd.copy()
            kpi_df = kpi_df.loc[kpi_df['kpi'].str.contains(kpi_name)]
            kpi_df['period'] = kpi_df.apply(lambda z: z.kpi.replace('{}_'.format(kpi_name), ''), axis=1)
            kpi_df.drop('kpi', axis=1, inplace=True)
            kpi_df.rename(columns={'value': kpi_name}, inplace=True)
            return kpi_df

        kpi_prod_tot = extract_period_reserve_kpi('PROD_TOT')
        kpi_res_up = extract_period_reserve_kpi('RES_UP_TOT')
        kpi_res_down = extract_period_reserve_kpi('RES_DOWN_TOT')
        df_q_r_tot = kpi_prod_tot.merge(kpi_res_up, how='inner', on='period')
        df_q_r_tot = df_q_r_tot.merge(kpi_res_down, how='inner', on='period')

        def extract_period_reserve_increment_kpi(kpi_name, l_exp=None):
            df = df_q_r_tot.copy()
            if l_exp:
                df[kpi_name] = df.apply(l_exp, axis=1)
            df = df[['period', kpi_name]]
            df.rename(columns={kpi_name: 'value'}, inplace=True)
            df['var'] = kpi_name
            return df

        df_q_r_dif_1 = extract_period_reserve_increment_kpi('P_R_UP', (lambda x: x.PROD_TOT + x.RES_UP_TOT))
        df_q_r_dif_2 = extract_period_reserve_increment_kpi('P_R_DOWN', (lambda x: x.PROD_TOT - x.RES_DOWN_TOT))
        df_q_r_dif_3 = extract_period_reserve_increment_kpi('PROD_TOT')
        df_q_r_dif = pd.concat([df_q_r_dif_1, df_q_r_dif_2, df_q_r_dif_3], ignore_index=True)

        return df_q_r_tot, df_q_r_dif

    @staticmethod
    def __prepare_production_kpi_df(df_P, df_P_rnwb):
        def aggregate_by_period(df_prod, type_name):
            df = df_prod.copy()
            df = df[['period', 'value']]
            df = df.groupby('period').sum().copy()
            df.reset_index(inplace=True)
            df['type'] = type_name
            return df

        df_p_agg_1 = aggregate_by_period(df_P, 'THERMAL')
        df_p_agg_2 = aggregate_by_period(df_P_rnwb, 'RENEWABLE')
        return pd.concat([df_p_agg_1, df_p_agg_2], ignore_index=True)

    def solve(self):
        if self.uc.solve():
            def prepare_output_df(sol_var_val_df):
                output_df = sol_var_val_df.copy().to_frame()
                output_df['unit'] = output_df.index.get_level_values('unit')
                output_df['period'] = output_df.index.get_level_values('period')
                output_df.columns = ['value', 'unit', 'period']
                output_df = output_df.reset_index(drop=True)
                output_df['period'] = output_df['period'].apply(lambda p: "{:03}".format(p))
                output_df = output_df[['period', 'unit', 'value']]
                return output_df

            df_thrm_prod = prepare_output_df(self.df_thrm_vars.p.apply(lambda z: z.solution_value))
            df_rnwb_prod = prepare_output_df(self.df_rnwb_vars.p_rnwb.apply(lambda z: z.solution_value))
            df_thrm_in_use = prepare_output_df(self.df_thrm_vars.in_use.apply(lambda z: z.solution_value))
            df_thrm_started = prepare_output_df(self.df_thrm_vars.on.apply(lambda z: z.solution_value))
            df_thrm_stopped = prepare_output_df(self.df_thrm_vars.off.apply(lambda z: z.solution_value))

            df_plan = self.__prepare_plan_df(df_thrm_in_use, df_thrm_started, df_thrm_stopped)

            all_kpis = [(kp.name, kp.compute()) for kp in self.uc.iter_kpis()]
            kpis_bd = pd.DataFrame(all_kpis, columns=['kpi', 'value'])
            kpis_bd = kpis_bd.append({'kpi': 'cplex time', 'value': "{:.3f}".format(self.uc.get_solve_details().time)}, ignore_index=True)

            kpis_cost = kpis_bd.loc[kpis_bd['kpi'].str.contains("Cost")].copy()
            kpis_cost['value'] = kpis_cost.apply(lambda x: round(x.value, 0), axis=1)
            kpis_plan = kpis_bd.loc[~kpis_bd['kpi'].str.contains("Cost")]

            df_q_r_tot, df_q_r_dif = self.__prepare_reserve_kpi_df(kpis_bd)

            kpis_bd = kpis_bd.loc[~kpis_bd['kpi'].str.contains("_TOT_")]

            df_P_agg = self.__prepare_production_kpi_df(df_thrm_prod, df_rnwb_prod)

            self._sc_outputs['thermal_prod'] = df_thrm_prod
            self._sc_outputs['renewable_prod'] = df_rnwb_prod
            self._sc_outputs['thermal_in_use'] = df_thrm_in_use
            self._sc_outputs['thermal_started'] = df_thrm_started
            self._sc_outputs['thermal_stopped'] = df_thrm_stopped
            self._sc_outputs['plan'] = df_plan
            self._sc_outputs['cost_kpis_high'] = kpis_cost.loc[(kpis_cost['kpi'].str.contains("Economic") | kpis_cost['kpi'].str.contains("Variable"))]
            self._sc_outputs['cost_kpis_low'] = kpis_cost.loc[(~kpis_cost['kpi'].str.contains("Economic") & ~kpis_cost['kpi'].str.contains("Variable"))]
            self._sc_outputs['plan_kpis'] = kpis_plan
            self._sc_outputs['all_kpis'] = kpis_bd
            self._sc_outputs['p_r_tot'] = df_q_r_tot
            self._sc_outputs['p_r_dif'] = df_q_r_dif
            self._sc_outputs['production'] = df_P_agg
            self._sc_outputs['production_rnwb'] = df_rnwb_prod

        else:
            all_kpis = [("Feasibility", 0)]
            kpis_bd = pd.DataFrame(all_kpis, columns=['kpi', 'value'])
            self._sc_outputs['kpis'] = kpis_bd
            
            from docplex.mp.conflict_refiner import ConflictRefiner
            infeas_cons = ConflictRefiner().refine_conflict(self.uc)
            print(infeas_cons)

    def get_model_info(self):
        return self.uc.print_information()

    def export_as_lp(self):
        return self.uc.export_as_lp()

    def get_outputs(self):
        return self._sc_outputs

    def get_sol_time(self):
        return self.uc.get_solve_details().time

    def end_model(self):
        self.uc.end()


## A Deterministic Optimization approach to UC

<a id=det></a>

Class `DeterministicUnitCommitment` implements the __Deterministic UC formulation__ of this use-case. It extends the formulation implemented in its base class by adding only two sets of constraints.

- Demand coverage constraints:<br><br>
$$\sum_{u\in U} p_{ut} \geq d_t \qquad \forall t\in T$$
where $d_t \in \mathbb{R}_+$ is the total energy demand load that the electrical system has to serve. 

- Energy reserve constraints:<br><br>
\begin{align}
\sum_{u \in U} \lambda^{\text{max}}_{ut} \cdot x^{\text{active}}_{ut} - p_{ut} &\geq r^\text{up}_t &\qquad\forall t \in T \\
\sum_{u \in U} p_{ut} - \lambda^{\text{min}}_{ut} \cdot x^{\text{active}}_{ut} &\geq r^\text{down}_t &\qquad\forall t \in T
\end{align}
where $r^\text{up}_t,\, r^\text{down}_t \in \mathbb{R}_+$ are constants which represent respectively the desired levels of up and down reserve for the system. 

In [2]:
class DeterministicUnitCommitment(UnitCommitment):

    def import_data(self, sc_inputs):
        super(DeterministicUnitCommitment, self).import_data(sc_inputs)

    def build_decision_vars(self):
        super(DeterministicUnitCommitment, self).build_decision_vars()

    def build_var_expressions(self):
        super(DeterministicUnitCommitment, self).build_var_expressions()

    def add_demand_cons(self):
        super(DeterministicUnitCommitment, self).add_demand_cons()
        for period, r in self.df_thrm_vars.groupby(level='period'):
            total_demand = self.df_load.D[period]
            self.uc.add_constraint(self.p_tot[period] >= total_demand)

    def add_spinning_reserve_cons(self):
        periods = self.df_load.index.values.tolist()
        for period in periods:
            total_reserve_up = self.df_load.RSS[period]
            total_reserve_down = self.df_load.RSB[period]
            ctname = "UPreserve_%d" % period
            self.uc.add_constraint(self.r_up_tot[period] >= total_reserve_up, ctname)
            self.uc.add_constraint(self.r_down_tot[period] >= total_reserve_down, ctname)

    def add_objective_and_kpis(self):
        super(DeterministicUnitCommitment, self).add_objective_and_kpis()

    def __init__(self, sc_inputs):
        super(DeterministicUnitCommitment, self).__init__(sc_inputs)
        self.add_spinning_reserve_cons()


## A Robust Optimization approach to UC

<a id=rob></a>

Class `RobustUnitCommitment` implements the __Robust UC formulation__ of this use-case. This approach considers the renewable energy generation to be uncertain. The power generated by renewable energy units in each period is modeled as a stochastic unknown, whose expected value and stardard deviation are given for each period. In particular, the Soyster's robust modeling technique is applied to obtain the formulation. Hence, the common abstract class is implemented by adding the following decision variables and constraints.


### Decision variables modeling the uncertainty of renewable energy production 

<a id=rob-vars></a>

In order to apply Soyster's robust formulation, the following continous non-negative variables are introduced for each renewable energy unit and for each period:<br><br>
$$ y_{ut} \in \mathbb{R}_+ \qquad \forall u\in U_W,\, t\in T $$


### Constraints modeling renewable energy variability in demand coverage and system reserve dimensioning 

<a id=rob-cons></a>

The following constrains have to be introduced in order to apply Soyster's robust formulation:<br><br>
\begin{align}
\sum_{u \in U} p_{ut} + \sum_{u \in U_W} y_{ut} &\geq d_t &\qquad\forall t \in T \\
\mu_{ut} - y_{ut} \leq p_{ut} &\leq \mu_{ut} + y_{ut} &\qquad\forall u \in U_W, t \in T \\
\sum_{u \in U_W} y_{ut} &\leq \max\left\{0, \sum_{u \in U_W} p_{ut} - \sigma_{ut}\right\} &\qquad\forall t \in T \\
\sum_{u \in U} \lambda^{\text{max}}_{ut} \cdot x^{\text{active}}_{ut} - p_{ut} &\geq \sum_{u \in U_W} \sigma_{ut} &\qquad\forall t \in T \\
\sum_{u \in U} p_{ut} - \lambda^{\text{min}}_{ut} \cdot x^{\text{active}}_{ut} &\geq \sum_{u \in U_W} \sigma_{ut} &\qquad\forall t \in T
\end{align}
where $\mu_{ut},\, \sigma_{ut}\in \mathbb{R}_+$ are respectively expected value and standard deviation of renewable power generation, for each renewable energy unit and for each period. 


### Business interpretation

<a id=rob-bus></a>

Some further comments follow, in order to motivate the formulation from a business perspective.
Variables $y_{ut}$ can be interpreted as *renewable energy under the effect of uncertainty*. Then, they make sure that the recommended planning remains feasible under possible fluctuations of the renewable generation. As a consequence, when $\sigma_{ut}$ is highly relevant respect to $\mu_{ut}$, reserve constraints (last two) become restrictive, hence thermal units have to burn more fuel to mitigate the effect of the uncertainty. On the other hand, when $\sigma_{ut}$ is less relevant respect to $\mu_{ut}$, the reserve constraints turn out to be less restrictive. Hence, the optimization has more chance to redistribute both the thermal and renewable energy productions by minimizing the cost, until the reserve constraints become tight. In other words, variables $y_{ut}$ try to increase safety when the renewable production's variability is relatively high, and increase sustainability as well when such variability is relatively low.

In [3]:
class RobustUnitCommitment(UnitCommitment):

    def import_data(self, sc_inputs):
        super(RobustUnitCommitment, self).import_data(sc_inputs)
        # Robust input
        self.df_renewable_forecast = sc_inputs['df_renewable_forecast']
        self.df_renewable_forecast.set_index(['unit', 'period'], inplace=True)

    def __build_robust_vars(self, units, periods, name=""):
        if name:
            name = "_{}".format(name)

        robust = self.uc.continuous_var_matrix(keys1=units, keys2=periods, name="robust{}".format(name))
        # Organize all decision variables in a DataFrame indexed by 'units' and 'periods'
        df_decision_vars_robust = pd.DataFrame({"robust{}".format(name): robust})
        # Set index names
        df_decision_vars_robust.index.names = ['unit', 'period']

        return df_decision_vars_robust

    def build_decision_vars(self):
        super(RobustUnitCommitment, self).build_decision_vars()
        wind_units = self.df_renewable_forecast.index.get_level_values('unit').unique().tolist()
        periods = self.df_load.index.values.tolist()
        self.df_robust_wind_vars = self.__build_robust_vars(wind_units, periods, "wind_prod")

    def build_var_expressions(self):
        super(RobustUnitCommitment, self).build_var_expressions()

    def add_demand_cons(self):
        super(RobustUnitCommitment, self).add_demand_cons()
        rwnb_groupby = self.df_rnwb_vars.groupby(level='period')
        robust_wind_groupby = self.df_robust_wind_vars.groupby(level='period')
        wind_units = self.df_renewable_forecast.index.get_level_values('unit').unique().tolist()

        for period, r in self.df_thrm_vars.groupby(level='period'):
            rnwb = rwnb_groupby.get_group(period)
            # demand coverage
            total_demand = self.df_load.D[period]
            rb_wind = robust_wind_groupby.get_group(period)
            self.uc.add_constraint(self.p_tot[period] + self.uc.sum(rb_wind.robust_wind_prod) >= total_demand)
            max_dev = sum([self.df_renewable_forecast.loc[unit, period].deviation for unit in wind_units])
            self.uc.add_constraint(self.r_up_tot[period] >= max_dev)
            self.uc.add_constraint(self.r_down_tot[period] >= max_dev)
            for unit in wind_units:
                self.uc.add_constraint(rb_wind.robust_wind_prod.loc[unit, period] + rnwb.p_rnwb.loc[unit, period] \
                                       >= self.df_renewable_forecast.loc[unit, period].forecast)
                self.uc.add_constraint(rnwb.p_rnwb.loc[unit, period] - rb_wind.robust_wind_prod.loc[unit, period] \
                                       <= self.df_renewable_forecast.loc[unit, period].forecast)

            self.uc.add_constraint(self.uc.sum(rb_wind.robust_wind_prod) <= \
                                   self.uc.max(0, self.uc.sum(rnwb.p_rnwb) - sum(
                                       self.df_renewable_forecast.loc[unit, period].deviation for unit in wind_units)))

    def add_objective_and_kpis(self):
        super(RobustUnitCommitment, self).add_objective_and_kpis()

    def __init__(self, sc_inputs):
        super(RobustUnitCommitment, self).__init__(sc_inputs)


## A Stochastic Optimization Approach to UC

<a id=sto></a>

Class `StochasticUnitCommitment` implements the __Stochastic UC formulation__ of this use-case. Analogously to the robust formulation discussed above, this approach considers the renewable energy generation to be uncertain, i.e. this is modeled as a stochastic unknown, whose expected value $\mu_{ut}$ and stardard deviation $\sigma_{ut}$ are given for each renewable unit $u \in U_W$ and for each period $t\in T$. In opposition to the robust approach, expected value and standard deviation of renewable generation are not used directly into the following formulation. However, they are used to simulate a certain number of scenarios by:
- projecting the renewable generation variability over the demand, 
- assuming that the realization of the stochastic unknown is normally distributed. 

As a consequence, the following stochastic formulation leads to the simultaneous optimization of multiple demand scenarios that should be covered by the system reserve. 


### Decision variables modeling the uncertainty of renewable energy production 

<a id=sto-vars></a>

The following continuous decision variables are introduced for each period and for each simulated scenario $s \in S$:<br><br>
$$z^{\text{up}}_{ts},\, z^{\text{down}}_{ts} \in \mathbb{R} \qquad\forall t \in T, s\in S$$


### Constraints modeling renewable energy variability in demand coverage and system reserve dimensioning 

<a id=sto-cons></a>

The following constraints are introduced:
\begin{align}
\sum_{u \in U} \lambda^{\text{max}}_{ut} \cdot x^{\text{active}}_{ut} + z^{\text{up}}_{ts} &= \delta_{ts} &\qquad\forall t\in T,\, s\in S\\
\sum_{u \in U} \left(\lambda^{\text{max}}_{ut} - \lambda^{\text{min}}_{ut}\right) \cdot x^{\text{active}}_{ut} + z^{\text{up}}_{ts} + z^{\text{down}}_{ts} &= 0 &\qquad\forall t\in T,\, s\in S\\
\frac{1}{|S|}\sum_{s \in S^\prime} \max\left\{0,\, z^{\text{up}}_{ts}\right\}  &\leq \omega^{\text{up}} &\qquad\forall t\in T\\
\frac{1}{|S|}\sum_{s \in S^\prime} \max\left\{0,\, z^{\text{down}}_{ts}\right\}  &\leq \omega^{\text{down}} &\qquad\forall t\in T
\end{align}
where: 
- $\delta_{ts} \in \mathbb{R}_+$ is the simulated load in period $t$ associated to scenario $s$;
- $S^\prime \subseteq S$ is a given subset of scenarios which has to be covered by the system reserve;
- $\omega^{\text{up}},\, \omega^{\text{down}}\in \mathbb{R}_+$ are constant parameters representing the desired maximum variation that has to be covered with respectively up and down reserve.

### Penalizing projected cost of reserve in the objective function

<a id=sto-obj></a>

Given parameters $\gamma^\text{up},\, \gamma^\text{down} > 0$, the following term:<br><br>
$$\gamma^\text{up} \sum_{t\in T}\sum_{s\in S} z^{\text{up}}_{ts} + \gamma^\text{down} \sum_{t\in T}\sum_{s\in S} z^{\text{down}}_{ts}$$
is added the objective function, in order to penalize the allocation of reserve.


### Business interpretation

<a id=sto-bus></a>

Some further comments follow, in order to motivate the formulation from a business perspective. 
Variables $z^{\text{up}}_{ts},\, z^{\text{down}}_{ts}$ can be interpreted as *corrections to the up and down reserve levels, which mitigate the effect of uncertainty*. Analogously to the case of robust formulation, these variables make sure that the recommended planning remains feasible under possible fluctuations of the renewable generation. In opposition to the robust approach, the introduced constraints ensure that the system reserve is able to cover a desired subset of scenarios, which represent simulated projections of the renewable generation variability over the demand load. Moreover, such corrections are tuned by targeting a desired mean value for the hourly system reserve, and penalizing local outliers with suitable costs.

In [4]:
class StochasticUnitCommitment(UnitCommitment):

    def import_data(self, sc_inputs):
        super(StochasticUnitCommitment, self).import_data(sc_inputs)
        ### Scenarios
        self.df_scenarios = sc_inputs['df_scenarios']
        self.df_scenarios.set_index(['scenario', 'period'], inplace=True)
        self.df_scenarios_conf = sc_inputs['df_scenarios_conf']
        # Stochastic parameters
        config = sc_inputs['config']
        if 'criteria' not in config.index:
            config.set_index('parameter', inplace=True)
        self.criteria = str(config.loc['criteria', 'value']).split()
        self.params = {}
        self.params['scenarios'] = self.df_scenarios.index.get_level_values('scenario').unique().tolist()
        self.params['prob_scen'] = float(1.0) / len(self.params['scenarios'])
        self.params['rf_num_scen_sru'] = int(len(self.params['scenarios']) * (1 - self.df_scenarios_conf.prob_sru[0]) + 1)
        self.params['rf_num_scen_srd'] = int(len(self.params['scenarios']) * (1 - self.df_scenarios_conf.prob_srd[0]) + 1)

    def build_stochastic_vars(self, periods):
        scenarios = self.params['scenarios']
        stoch_res_up = self.uc.continuous_var_matrix(keys1=scenarios, keys2=periods, lb=-self.uc.infinity, name="stoch_res_up")
        stoch_res_down = self.uc.continuous_var_matrix(keys1=scenarios, keys2=periods, lb=-self.uc.infinity, name="stoch_res_down")
        df_decision_vars_scen = pd.DataFrame({'stoch_res_up': stoch_res_up, 'stoch_res_down': stoch_res_down})
        df_decision_vars_scen.index.names = ['scenario', 'period']

        return df_decision_vars_scen

    def build_decision_vars(self):
        super(StochasticUnitCommitment, self).build_decision_vars()
        periods = self.df_load.index.values.tolist()
        self.df_scenario_vars = self.build_stochastic_vars(periods)

    def build_var_expressions(self):
        super(StochasticUnitCommitment, self).build_var_expressions()
        periods = self.df_load.index.values.tolist()
        self.stoch_res_up_tot = {}
        self.stoch_res_down_tot = {}
        self.stoch_res_p_up_tot = {}
        self.stoch_res_p_down_tot = {}
        df_join_scenario_vars_up = self.df_scenario_vars.join(self.df_scenarios, how='inner')
        for period in periods:
            sru_expr_list = [self.uc.max(df_join_scenario_vars_up.stoch_res_up[s, period], 0) for s in self.params['scenarios']]
            self.stoch_res_up_tot[period] = self.params['prob_scen'] * self.uc.sum(sru_expr_list)
            srd_expr_list = [self.uc.max(df_join_scenario_vars_up.stoch_res_down[s, period], 0) for s in self.params['scenarios']]
            self.stoch_res_down_tot[period] = self.params['prob_scen'] * self.uc.sum(srd_expr_list)

            if 'RiskFactor' in self.criteria:
                n_scen = len(self.params['scenarios'])
                n_scen_p_sru = self.params['rf_num_scen_sru']
                n_scen_p_srd = self.params['rf_num_scen_srd']
                sru_expr_list_p = [self.uc.max(df_join_scenario_vars_up.stoch_res_up[s, period], 0) for s in range(n_scen - n_scen_p_sru, n_scen)]
                self.stoch_res_p_up_tot[period] = self.params['prob_scen'] * self.uc.sum(sru_expr_list_p)
                srd_expr_list_p = [self.uc.max(df_join_scenario_vars_up.stoch_res_down[s, period], 0) for s in range(n_scen - n_scen_p_srd, n_scen)]
                self.stoch_res_p_down_tot[period] = self.params['prob_scen'] * self.uc.sum(srd_expr_list_p)

    def add_demand_cons(self):
        super(StochasticUnitCommitment, self).add_demand_cons()
        for period, r in self.df_thrm_vars.groupby(level='period'):
            total_demand = self.df_load.D[period]
            self.uc.add_constraint(self.p_tot[period] >= total_demand)

    def add_stoch_model_cons(self):
        df_join_scenario_vars_up = self.df_scenario_vars.join(self.df_scenarios, how='inner')
        for scenario in self.params['scenarios']:
            for period, r in df_join_scenario_vars_up.groupby(level='period'):
                self.uc.add_constraint(
                    self.p_tot[period] + self.r_up_tot[period] + r.stoch_res_up[scenario, period] \
                    == r.load[scenario, period])
                self.uc.add_constraint(
                    r.stoch_res_up[scenario, period] + r.stoch_res_down[scenario, period] \
                    + self.r_up_tot[period] + self.r_down_tot[period] == 0)

        periods = self.df_load.index.values.tolist()
        for period in periods:
            if 'MaxCoverage' in self.criteria:
                self.uc.add_constraint(self.stoch_res_up_tot[period] <= self.df_scenarios_conf.sru_mc[0])
                self.uc.add_constraint(self.stoch_res_down_tot[period] <= self.df_scenarios_conf.srd_mc[0])
            if 'RiskFactor' in self.criteria:
                self.uc.add_constraint(self.stoch_res_p_up_tot[period] <= self.df_scenarios_conf.sru_rf[0])
                self.uc.add_constraint(self.stoch_res_p_down_tot[period] <= self.df_scenarios_conf.srd_rf[0])

    def add_objective_and_kpis(self):
        super(StochasticUnitCommitment, self).add_objective_and_kpis()
        sru_cost = self.uc.sum([self.df_scenarios_conf.price_sru[0] * i for i in self.stoch_res_up_tot.values()])
        srd_cost = self.uc.sum([self.df_scenarios_conf.price_srd[0] * i for i in self.stoch_res_down_tot.values()])
        self.obj += sru_cost + srd_cost
        self.uc.minimize(self.obj)
        self.uc.add_kpi(sru_cost, "Stoch Vars Res Up Value")
        self.uc.add_kpi(srd_cost, "Stoch Vars Res Down Value")

    def __init__(self, sc_inputs):
        super(StochasticUnitCommitment, self).__init__(sc_inputs)
        self.add_stoch_model_cons()


In [ ]:
### Main function

config = None
if 'config' not in inputs or inputs['config'].empty:
    config = pd.DataFrame(data={'parameter': ['model'], 'value': ['D']})
else:
    config = inputs['config']
config.set_index('parameter', inplace=True)
    
uc = None
if config.loc['model', 'value'] == 'D':
    uc = DeterministicUnitCommitment(inputs)
elif config.loc['model', 'value'] == 'R':
    uc = RobustUnitCommitment(inputs)
elif config.loc['model', 'value'] == 'S':
    uc = StochasticUnitCommitment(inputs)

if uc:
    uc.solve()
    outputs = uc.get_outputs()

### Authors

- __Gianmaria Leo__ is a Senior Operations Research Engineer and Data Scientist with Data Science Elite team (IBM Analytics).
- __Sumit K Bose__ is a Senior Managing Consultant and Data Scientist with Data Science Elite team (IBM Analytics).

Copyright © IBM Corp. 2018. Released as licensed Sample Materials.